<a href="https://colab.research.google.com/github/lalopezabc123/csv/blob/main/Proyecci%C3%B3n_del_N%C3%BAmero_de_asegurados_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Previsión del Número de asegurados activos que ingresaran al sistema usando PyCaret**


## Para quitar warnings y actualizando pip para instalar correctamente pycaret

In [ ]:
import warnings
warnings.filterwarnings('ignore')
!pip install --upgrade pip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


##Instalacion de PyCaret

In [ ]:

!pip install pycaret


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


##Importando librerias numpy y pandas para carga de datos y transformaciones

In [ ]:
import numpy as np 
import pandas as pd 

## Cargando dataset

In [ ]:
data  = pd.read_csv("https://raw.githubusercontent.com/lalopezabc123/csv/main/08-12-2022BD.csv")


In [ ]:
data

,Fecha,Año,Mes,dia,Trimestre,Dia_semana,Semana,Dia_año,Numero de Asegurados
0,2012-01-01,2012,1,1,1,1,1,1,603160
1,2012-02-01,2012,2,1,1,4,5,4,612813
2,2012-03-01,2012,3,1,1,5,9,5,616574
3,2012-04-01,2012,4,1,2,1,14,1,616036
4,2012-05-01,2012,5,1,2,3,18,3,626015
...,...,...,...,...,...,...,...,...,...
127,2022-08-01,2022,8,1,3,2,32,2,758571
128,2022-09-01,2022,9,1,3,5,36,5,761201
129,2022-10-01,2022,10,1,4,7,40,7,763805
130,2022-11-01,2022,11,1,4,3,45,3,766387


## Transformación de tipo de datos

In [ ]:
data["datetime"] = pd.to_datetime(data["Fecha"])
data["Numero de Asegurados"] = pd.to_numeric(data["Numero de Asegurados"])

## Verificando ultimos 10 registros

In [ ]:
data.tail(10)

,Fecha,Año,Mes,dia,Trimestre,Dia_semana,Semana,Dia_año,Numero de Asegurados,datetime
122,2022-03-01,2022,3,1,1,3,10,3,744934,2022-03-01
123,2022-04-01,2022,4,1,2,6,14,6,741484,2022-04-01
124,2022-05-01,2022,5,1,2,1,19,1,741490,2022-05-01
125,2022-06-01,2022,6,1,2,4,23,4,753201,2022-06-01
126,2022-07-01,2022,7,1,3,6,27,6,755908,2022-07-01
127,2022-08-01,2022,8,1,3,2,32,2,758571,2022-08-01
128,2022-09-01,2022,9,1,3,5,36,5,761201,2022-09-01
129,2022-10-01,2022,10,1,4,7,40,7,763805,2022-10-01
130,2022-11-01,2022,11,1,4,3,45,3,766387,2022-11-01
131,2022-12-01,2022,12,1,4,5,49,5,768950,2022-12-01


## Gráfica para explorar comportamiento de lo datos



In [ ]:
import plotly.express as px 
fig = px.line(data, x="Fecha", y="Numero de Asegurados")
fig.show()

## Importando pycaret

In [ ]:
from pycaret.regression import *

## Configurando parametros para modelo de series de tiempo

In [ ]:
s = setup(data, target = "Numero de Asegurados", train_size = 0.95,
              data_split_shuffle = False, fold_strategy = 'timeseries', fold = 3,
              ignore_features = ["Fecha", "datetime"],
              numeric_features = ["Dia_año", "Año", "Mes", "dia"],
              categorical_features = ["Mes"],
              silent = True, verbose = False, session_id = 123)

INFO:logs:PyCaret Supervised Module
INFO:logs:ML Usecase: regression
INFO:logs:version 2.3.10
INFO:logs:Initializing setup()
INFO:logs:setup(target=Numero de Asegurados, ml_usecase=regression, available_plots={'parameter': 'Hyperparameters', 'residuals': 'Residuals', 'error': 'Prediction Error', 'cooks': 'Cooks Distance', 'rfe': 'Feature Selection', 'learning': 'Learning Curve', 'manifold': 'Manifold Learning', 'vc': 'Validation Curve', 'feature': 'Feature Importance', 'feature_all': 'Feature Importance (All)', 'tree': 'Decision Tree', 'residuals_interactive': 'Interactive Residuals'}, train_size=0.95, test_data=None, preprocess=True, imputation_type=simple, iterative_imputation_iters=5, categorical_features=['Mes'], categorical_imputation=constant, categorical_iterative_imputer=lightgbm, ordinal_features=None, high_cardinality_features=None, high_cardinality_method=frequency, numeric_features=['Dia_año', 'Año', 'Mes', 'dia'], numeric_imputation=mean, numeric_iterative_imputer=lightgbm

## Comparación de algoritmos

In [ ]:
best = compare_models(sort = 'MAE')

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
dt,Decision Tree Regressor,50486.9677,4.322880e+09,59043.7273,-1.2403,0.0798,0.0661,0.0200
gbr,Gradient Boosting Regressor,58453.0145,5.340763e+09,68087.2880,-1.9583,0.0917,0.0775,0.0533
et,Extra Trees Regressor,60349.1740,5.625445e+09,69503.8243,-2.1260,0.0932,0.0802,0.1900
rf,Random Forest Regressor,62024.7677,5.944959e+09,71473.3531,-2.3494,0.0956,0.0826,0.2433
ada,AdaBoost Regressor,64562.1695,6.622359e+09,74588.2257,-2.7473,0.0992,0.0862,0.0933
huber,Huber Regressor,65231.2866,7.334426e+09,71699.4177,-2.5305,0.0986,0.0850,0.0467
dummy,Dummy Regressor,66881.2430,7.481925e+09,73196.1841,-2.7040,0.1007,0.0875,0.0267
knn,K Neighbors Regressor,78370.2430,8.928267e+09,89112.8781,-4.1283,0.1216,0.1051,0.0267
lightgbm,Light Gradient Boosting Machine,79438.8967,9.520903e+09,87450.2353,-3.9737,0.1188,0.1056,0.2867
en,Elastic Net,104839.7097,1.453666e+10,113616.7265,-9.5554,0.1473,0.1454,0.0200


INFO:logs:create_model_container: 18
INFO:logs:master_model_container: 18
INFO:logs:display_container: 2
INFO:logs:DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=123, splitter='best')
INFO:logs:compare_models() succesfully completed......................................


## Seleccion de algoritmo para modelo

In [ ]:
best_model = create_model('dt', verbose = False)
f = finalize_model(best_model)

INFO:logs:Initializing create_model()
INFO:logs:create_model(estimator=dt, fold=None, round=4, cross_validation=True, predict=True, fit_kwargs=None, groups=None, refit=True, verbose=False, system=True, metrics=None, experiment_custom_tags=None, add_to_model_list=True, probability_threshold=None, display=None, return_train_score=False, kwargs={})
INFO:logs:Checking exceptions
INFO:logs:Importing libraries
INFO:logs:Copying training dataset
INFO:logs:Defining folds
INFO:logs:Declaring metric variables
INFO:logs:Importing untrained model
INFO:logs:Decision Tree Regressor Imported succesfully
INFO:logs:Starting cross validation
INFO:logs:Cross validating with TimeSeriesSplit(max_train_size=None, n_splits=3), n_jobs=-1
INFO:logs:Calculating mean and std
INFO:logs:Creating metrics dataframe
INFO:logs:Finalizing model
INFO:logs:Uploading results into container
INFO:logs:Uploading model into container now
INFO:logs:create_model_container: 20
INFO:logs:master_model_container: 20
INFO:logs:displ

## Preparando dataframe para predicción

In [ ]:
all_dates = pd.date_range(start="2012-01-01", end = "2026-12-31", freq = 'D')
# dataframe vacio
score_df = pd.DataFrame()
# añadiendo columnas al dataset
score_df["Fecha"] = all_dates
score_df["datetime"] = pd.to_datetime(score_df["Fecha"])
score_df["Año"] = [i.year for i in score_df["Fecha"]]
score_df["Mes"] = [i.month for i in score_df["Fecha"]]
score_df["dia"] = [i.day for i in score_df["Fecha"]]
score_df["Dia_año"] = [i.dayofyear for i in score_df["Fecha"]]
score_df["Dia_semana"] = [i.dayofweek for i in score_df["Fecha"]]
score_df["Trimestre"] = [i.quarter for i in score_df["Fecha"]]
score_df["Semana"] = [i.week for i in score_df["Fecha"]] 

p = predict_model(best_model, data = score_df)   

NameError: ignored

## Filtrando dataframes para graficar

In [ ]:
data = data[data['datetime'] <= "2022-12-01"]
p = p[p['datetime'] > "2022-12-01"]

## Grafica proyección

In [ ]:
import plotly.graph_objects as go
fig = go.Figure()

fig.add_trace(go.Scatter(x = data["Fecha"], y = data["Numero de Asegurados"], mode='lines', name='Histórico'))
fig.add_trace(go.Scatter(x = p["Fecha"], y = p["Label"], mode = 'lines', name='Proyección'))
fig.show()

# Conclusiones

Para este análisis hemos considerado información del numero de asegurados desde el año (2012-2022) del INIDE (Sistema de Estadistica PUblicas y Censo),Nota. para el año 2022 estimamos el mejor cierre del mes dic-2022



Para que el modelo tome sentido hemos hecho transformaciones en la fecha, obteniendo información que entrene a nuestro modelo como el trimestre, semana, dia del año.

Nuestro objetivo es predecir el comportamiento de los proximos días de la cosecha del ciclo en curso y el ciclo siguiente completo, de manera que se puedan realizar las actividades propias de la naturaleza del negocio, de esta manera se evalua la liquidez, las necesidades de personal y de instalaciones, los insumos y quehacer logístico.

Nuestros datos se caracterizan por tener estacionalidad, puesto que el periodo de recoleccion del café aproximadamente de Septiembre a Marzo, el modelo ha predicho correctamente que fuera de ese periodo no habrá ninguna entrada.

Podemos concluir que en el periodos de interés (2022-2023) se aprecia un comportamiento similar al del ciclo anterior (2021-2022).